![Wherobots logo](../assets/img/header-logo.png)

# WherobotsAI Raster Inference (Text-Prompted Models)

In this notebook we introduce text-prompted Raster Inference in WherobotsAI, powered by Meta AI’s SAM2 and Google DeepMind’s OWLv2 models. Raster Inference brings the ability to run advanced vision models directly on large-scale satellite imagery, turning pixels into analysis-ready vector features at scale.  

We will demonstrate how to apply these models to NAIP imagery of Miami Airport, using `RS_Text_to_BBoxes` (OWLv2) for object detection and `RS_Text_to_Segments` (SAM2) for segmentation.  

[Read more about Wherobots Raster Inference in the documentation](https://docs.wherobots.com/latest/tutorials/wherobotsai/wherobots-inference/raster-inference-overview/?h=raster+inference).  

## Define Sedona context

In [ ]:
from sedona.spark import *
from pyspark.sql.functions import expr

config = SedonaContext.builder().getOrCreate()
sedona = SedonaContext.create(config)

## Load NAIP imagery for Miami Airport

We will use the native raster reader to load GeoTIFFs as out-of-database or "out-db" rasters and perform dyanamic tiling on read.
Spliting the large GeoTIFF into small tiles improves the distribution of workload across the cluster.

In [ ]:
url = "s3://wherobots-examples/data/naip/miami-airport.tiff"
tile_size = 256

df = sedona.read.format("raster")\
        .option("tileWidth", tile_size)\
        .option("tileHeight", tile_size)\
        .load(url)

df.createOrReplaceTempView("df")
df.show(5)

## Viewing Raster Inputs

Before running inference, it’s useful to explore the imagery itself.  
We’ll visualize the footprints of the raster tiles with SedonaKepler and preview a few raw images using SedonaUtils. This gives us confidence that the data is being read correctly and aligned spatially before applying any models.

> Tip: You can also save the Kepler map as an interactive HTML file with `kepler_map.save_to_html()`.

In [ ]:
kepler_map = SedonaKepler.create_map()
df = df.withColumn('footprint', expr("ST_TRANSFORM(RS_CONVEXHULL(rast),'EPSG:4326')"))
SedonaKepler.add_df(kepler_map, df=df, name="Image Footprints")

kepler_map

In [ ]:
htmlDf = sedona.sql(f"""SELECT RS_AsImage(rast, 250) as FROM df limit 5""")
SedonaUtils.display_image(htmlDf)

## Running segementation using SAM2 model

Now we’ll run inference over the raster tiles using Wherobots SQL function `RS_Text_to_Segments()`.

For this example, we specify the following parameters -
- Model: `"sam2"`
- Text prompt: `"airplanes"`
- Confidence threshold: `0.5`

The confidence threshold controls which detections are returned (scores range from 0 to 1).  
For this particular model, most positives have confidence scores of at most ~0.7, so we start with `0.5` to favor **recall** on this model/dataset. This helps us understand how the model performs before tightening the threshold later.

The function returns predicted segments for each raster in our region of interest.
We’ll cache the results for efficiency and register them as a temporary view so we can explore them further.

In [ ]:
model_id = "sam2"
prompt = "airplanes"
threshold = 0.5

preds = sedona.sql(f"""
    SELECT 
        rast, 
        RS_TEXT_TO_SEGMENTS(
            '{model_id}', 
            rast, 
            '{prompt}', 
            {threshold}
        ) AS preds 
    FROM df
""")

preds.cache().count()
preds.createOrReplaceTempView("preds")

### Preparing Results

The raw inference output contains arrays of predictions (segments, confidence scores, and labels) for each raster tile.  
Before plotting, we need to flatten these arrays so that each row corresponds to a single prediction.

We’ll:  
1. Filter out empty or invalid results.  
2. Use `arrays_zip` + `explode` to turn lists of predictions into individual rows.  
3. Keep the geometry, confidence score, and label for each detected object.  

This prepares the data for mapping and analysis with SedonaKepler.

In [ ]:
preds_filtered = sedona.sql(f"""
  SELECT *
  FROM preds
  WHERE
    size(preds.labels) > 0
    AND array_contains(preds.labels, 1)
    AND NOT array_contains(preds.segments_wkt, 'POLYGON EMPTY')
""")

preds_filtered.createOrReplaceTempView("preds_filtered")
preds_filtered.show(5)

In [ ]:
exploded = sedona.sql("""
    SELECT
        rast,
        exploded_predictions.*
    FROM
        preds_filtered
    LATERAL VIEW explode(arrays_zip(preds.segments_wkt, preds.confidence_scores, preds.labels)) AS exploded_predictions
    WHERE
        exploded_predictions.confidence_scores != 0.0
""")
exploded.cache().count()
exploded.createOrReplaceTempView("exploded")
exploded.show(5)

### Viewing segmentation results

Let’s map the predicted segments to sanity-check geometry and coverage.
We’ll add the detections as a layer in SedonaKepler—use the tooltip to inspect confidence per feature.

In [ ]:
kepler_map = SedonaKepler.create_map()
SedonaKepler.add_df(kepler_map, df=exploded, name="Airplane Detections")

kepler_map

### Overlay results on source imagery

To see detections on top of the underlying rasters, use `show_detections`.
It expects the non-exploded predictions (arrays per tile) and can filter by confidence.

> Tip: show_detections works with DataFrames that still have the raster column and arrays of results; exploded DataFrames aren’t supported.


In [ ]:
unpacked_preds_df = sedona.sql("""
  SELECT
    rast,
    preds.segments_wkt,
    preds.confidence_scores,
    preds.labels
  FROM preds_filtered
""")

In [ ]:
from wherobots.inference.plot.detections import show_detections

show_detections(
    unpacked_preds_df,
    confidence_threshold=0.7,
    plot_geoms=True,
    geometry_column="segments_wkt",
)

## Running Object Detection using OWLv2 model

Now we’ll run inference over the raster tiles using Wherobots’ SQL function `RS_Text_to_BBoxes()`.

For this example, we specify the following parameters –
- Model: `"owlv2"`
- Text prompt: `"airplanes"`
- Confidence threshold: `0.5`

The function returns predicted bounding boxes for each raster in our region of interest.

We’ll cache the results for efficiency and register them as a temporary view so we can explore them further.

In [ ]:
model_id = "owlv2"
prompt = "airplanes"
threshold = 0.5

preds = sedona.sql(f"""
    SELECT 
        rast, 
        RS_TEXT_TO_BBoxes(
            '{model_id}', 
            rast, 
            '{prompt}', 
            {threshold}
        ) AS preds 
    FROM df
""")
preds.cache().count()
preds.createOrReplaceTempView("preds")

### Preparing Results

The raw inference output contains arrays of predictions (bounding boxes, confidence scores, and labels) for each raster tile.  
Before plotting, we need to flatten these arrays so that each row corresponds to a single prediction.

We’ll:  
1. Filter out empty or invalid results.  
2. Use `arrays_zip` + `explode` to turn lists of predictions into individual rows.  
3. Keep the geometry, confidence score, and label for each detected object.  

This prepares the data for mapping and analysis with SedonaKepler.

In [ ]:
preds_filtered = sedona.sql(f"""
  SELECT *
  FROM preds
  WHERE
    size(preds.labels) > 0
    AND array_contains(preds.labels, 1)
    AND NOT array_contains(preds.bboxes_wkt, 'POLYGON EMPTY')
""")
preds_filtered.createOrReplaceTempView("preds_filtered")
preds_filtered.show()

Each raster tile returns multiple predictions as arrays.  
We use `explode` to flatten these arrays so every detected object becomes its own row for easier mapping and analysis.

In [ ]:
exploded = sedona.sql("""
    SELECT
        rast,
        exploded_predictions.*
    FROM
        preds_filtered
    LATERAL VIEW explode(arrays_zip(preds.bboxes_wkt, preds.confidence_scores, preds.labels)) AS exploded_predictions
    WHERE
        exploded_predictions.confidence_scores != 0.0
""")
exploded.cache().count()
exploded.createOrReplaceTempView("exploded")
exploded.show()

### Viewing Object Detection results

Let’s map the predicted Airplane bboxes to sanity-check geometry and coverage.
We’ll add the detections as a layer in SedonaKepler—use the tooltip to inspect confidence per feature.

In [ ]:
kepler_map = SedonaKepler.create_map()
SedonaKepler.add_df(kepler_map, df=exploded, name="Airplane Detections")

kepler_map

### Overlay results on source imagery

To see detections on top of the underlying rasters, use `show_detections`.
It expects the non-exploded predictions (arrays per tile) and can filter by confidence.

> Tip: show_detections works with DataFrames that still have the raster column and arrays of results; exploded DataFrames aren’t supported.

In [ ]:
unpacked_preds_df = sedona.sql("""
  SELECT
    rast,
    preds.bboxes_wkt,
    preds.confidence_scores,
    preds.labels
  FROM preds_filtered
""")

We see below that OWLv2 and SAM2 do remarkably well at identifying airplanes with little user effort! Previously, achieving similar results was a significant undertaking. An entire Machine Learning engineering team would have needed to build such a model from scratch.

In [ ]:
from wherobots.inference.plot.detections import show_detections

show_detections(
    unpacked_preds_df,
    confidence_threshold=0.5,
    plot_geoms=True,
    side_by_side=False,
    geometry_column="bboxes_wkt",
)

## Next Steps with Raster Inference

With access to general-purpose, text-promptable models, what will you predict and georeference next?

Some ideas on next steps to try, include:

* Predicting different objects next to the airplanes in the image tiles above using new text prompts.
* Adjusting the confidence score threshold for `RS_Text_to_Segments` or `RS_Text_to_BBoxes` to see how SAM2 or OWLv2 respond.
* Loading a new imagery dataset with our [STAC Reader](https://docs.wherobots.com/latest-snapshot/references/wherobotsdb/vector-data/Stac/) and try to predict a different feature of interest, such as agriculture, buildings, or tree crowns.

We're excited to hear about what you're doing with SAM2 and OWLv2! 